In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/датасет_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6269242 entries, 0 to 6269241
Data columns (total 10 columns):
 #   Column                            Dtype 
---  ------                            ----- 
 0   Дата и время события              object
 1   Экран                             object
 2   Функционал                        object
 3   Действие                          object
 4   Идентификатор устройства          int64 
 5   Номер сессии в рамках устройства  int64 
 6   Производитель устройства          object
 7   Модель устройства                 object
 8   Тип устройства                    object
 9   ОС                                object
dtypes: int64(2), object(8)
memory usage: 478.3+ MB


In [3]:
df.describe()

,Идентификатор устройства,Номер сессии в рамках устройства
count,6.269242e+06,6.269242e+06
mean,5.940995e+04,1.000000e+10
std,4.575802e+04,8.478572e+02
min,1.000000e+00,1.000000e+10
25%,2.255000e+04,1.000000e+10
50%,4.795450e+04,1.000000e+10
75%,8.854300e+04,1.000000e+10
max,1.803410e+05,1.000002e+10


In [5]:
df[df['Идентификатор устройства']==339]

,Дата и время события,Экран,Функционал,Действие,Идентификатор устройства,Номер сессии в рамках устройства,Производитель устройства,Модель устройства,Тип устройства,ОС
0,2025-09-29T10:20:27+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000009,Redmi,Redmi Note 12,phone,Android
1,2025-09-29T10:20:27+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000009,Redmi,Redmi Note 12,phone,Android
2,2025-09-29T10:21:56+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000009,Redmi,Redmi Note 12,phone,Android
3,2025-09-29T10:21:56+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000009,Redmi,Redmi Note 12,phone,Android
4,2025-09-29T10:22:01+03:00[Europe/Moscow],Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',339,10000000009,Redmi,Redmi Note 12,phone,Android
...,...,...,...,...,...,...,...,...,...,...
4146030,2025-10-30T16:04:31+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000026,Redmi,Redmi Note 12,phone,Android
5449230,2025-10-16T10:45:15+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000019,Redmi,Redmi Note 12,phone,Android
5449231,2025-10-16T10:45:15+03:00[Europe/Moscow],Еще,Открытие экрана,NaN,339,10000000019,Redmi,Redmi Note 12,phone,Android
5449232,2025-10-16T10:45:19+03:00[Europe/Moscow],Еще,Переход в раздел 'Приборы учета',Тап на кнопку 'Приборы учёта',339,10000000019,Redmi,Redmi Note 12,phone,Android
